In [1]:
import numpy as np
import pandas as pd
import nltk
import re

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aonno\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [40]:
df = pd.read_csv("sample.csv")

In [41]:
df.head()

,article_id,article_text
0,1,In the northern part of Austin there once dwel...


In [42]:
df['article_text'][0]

"In the northern part of Austin there once dwelt an honest family by the name of Smothers. The family consisted of John Smothers, his wife, himself, their little daughter, five years of age, and her parents, making six people toward the population of the city when counted for a special write-up, but only three by actual count.\r\n\r\nOne night after supper the little girl was seized with a severe colic, and John Smothers hurried down town to get some medicine.\r\n\r\nHe never came back.\r\n\r\nThe little girl recovered and in time grew up to womanhood.\r\n\r\nThe mother grieved very much over her husband's disappearance, and it was nearly three months before she married again, and moved to San Antonio.\r\n\r\nThe little girl also married in time, and after a few years had rolled around, she also had a little girl five years of age.\r\n\r\nShe still lived in the same house where they dwelt when her father had left and never returned.\r\n\r\nOne night by a remarkable coincidence her litt

In [43]:
from nltk.tokenize import sent_tokenize
sentences = []
for s in df['article_text']:
    sentences.append(sent_tokenize(s))

#print(sentences, "\n\n")

sentences = [y for x in sentences for y in x]

#print(sentences)

In [44]:
sentences[:5]

['In the northern part of Austin there once dwelt an honest family by the name of Smothers.',
 'The family consisted of John Smothers, his wife, himself, their little daughter, five years of age, and her parents, making six people toward the population of the city when counted for a special write-up, but only three by actual count.',
 'One night after supper the little girl was seized with a severe colic, and John Smothers hurried down town to get some medicine.',
 'He never came back.',
 'The little girl recovered and in time grew up to womanhood.']

In [45]:
word_embeddings = {}
f = open('glove.6B/glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs

f.close()

In [46]:
len(word_embeddings)

400000

In [47]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [48]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aonno\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [49]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [50]:
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [51]:
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [52]:
# extracting sentence vectors
sentence_vectors = []

for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/ (len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    
    sentence_vectors.append(v)

print(sentence_vectors)

[array([-3.12934607e-01, -2.09904492e-01,  1.93016469e-01, -1.21851377e-01,
       -1.53727233e-04,  4.27783638e-01, -2.22252086e-01,  8.82544741e-02,
       -8.23534578e-02,  9.95338708e-03, -7.86351562e-02, -1.36250839e-01,
        3.21631014e-01,  2.07077980e-01, -2.00036258e-01, -2.62659669e-01,
        3.91281694e-01, -4.07267921e-02, -2.64441937e-01,  3.97384971e-01,
        3.29687536e-01,  5.26326783e-02,  1.99849508e-03, -7.27812797e-02,
        6.81690574e-02,  1.65216830e-02, -1.26283079e-01, -1.98668152e-01,
       -6.84421882e-02,  6.06371686e-02,  3.97639088e-02,  3.90862346e-01,
        4.15649652e-01,  6.71165669e-03,  4.67566028e-03,  1.53675288e-01,
        1.77631900e-01,  2.10341573e-01,  1.21845387e-01, -4.14240696e-02,
       -2.13213578e-01, -1.72459826e-01,  1.71451688e-01, -6.53278455e-02,
       -1.67969108e-01,  5.54451831e-02,  2.07089841e-01, -3.15180570e-02,
        1.13269590e-01, -2.84731388e-01, -2.42820755e-01,  1.25934777e-03,
        4.86281931e-01, 

In [53]:
#print(word_embeddings)

In [54]:
# creating a similarity matrix using cosine similarity
sim_mat = np.zeros([len(sentences), len(sentences)])

In [55]:
from sklearn.metrics.pairwise import cosine_similarity

In [56]:
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [57]:
#!pip install networkx

In [58]:
## applying pagerank
import networkx as nx
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [59]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [60]:
for i in range(10):
    print(ranked_sentences[i][1])

The little girl also married in time, and after a few years had rolled around, she also had a little girl five years of age.
After a little Pansy seemed to grow worse, and John Smith again attempted to go for medicine, but his wife would not let him.
One night by a remarkable coincidence her little girl was taken with cramp colic on the anniversary of the disappearance of John Smothers, who would now have been her grandfather if he had been alive and had a steady job.
'I will go downtown and get some medicine for her,' said John Smith (for it was none other than he whom she had married).
One night after supper the little girl was seized with a severe colic, and John Smothers hurried down town to get some medicine.
The family consisted of John Smothers, his wife, himself, their little daughter, five years of age, and her parents, making six people toward the population of the city when counted for a special write-up, but only three by actual count.
She still lived in the same house wher